# 05 - JSON mode

JSON mode in OpenAI's chat completion allows developers to receive structured data responses in JSON format. 
This feature is particularly useful for applications that require precise data parsing and integration with other systems. By specifying a JSON schema, developers can guide the model to generate responses that adhere to a predefined structure, ensuring consistency and ease of use when handling complex data interactions. This approach enhances the model's utility in scenarios like API development, data processing, and automated workflows, providing a more reliable and structured output.

First, we use the `import` statement to let our application know that we're going to be using the OpenAI library in our code.

In [ ]:
%pip install openai

We'll create a new `AzureOpenAI` object and pass in the API key and version and the endpoint URL to be used.

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION")
)
def print_response(system_prompt, user_prompt):
    response = client.chat.completions.create(
        model = os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        messages = [{"role" : "system", "content" : system_prompt}, {"role" : "user", "content" : user_prompt}],
    )
    print(response.choices[0].message.content)

## Send a prompt to Azure OpenAI using the OpenAI library

Now that we have defined an Azure OpenAI instance, let's try a Chat Completion. We'll call the `chat.completions.create()` method. Note that for the `model` value, we actually pass in the id of our Azure OpenAI `deployment`. We'll also pass the `prompt` we want to use as the `content` of the `messages` parameter.

* Exercise: Write a prompt that generates the expected completion
* Input text:
  ```
  Hello, my name is Mateo Gomez. I lost my Credit card on August 17th, and I would like to request its cancellation. The last purchase I made was of a Chicken parmigiana dish at Contoso Restaurant, located near the Hollywood Museum, for $40. Below is my personal information for validation:
  Profession: Accountant
  Social Security number is 123-45-6789
  Date of birth: 9-9-1989
  Phone number: 949-555-0110
  Personal address: 1234 Hollywood Boulevard Los Angeles CA
  Linked email account: mateo@contosorestaurant.com
  Swift code: CHASUS33XXX
  ```
* Expected completion:
  ```
  {
      "reason": "Lost card",
      "classified_reason": "lost_card",
      "name": "Mateo Gomez",
      "ssn": "123-45-6789",
      "dob": "09/09/1989"
  }
  ```

In [ ]:
system_prompt ="""
Given an input text containing a customer's personal information and request details, extract and structure the relevant information into a JSON object. 
The object should include the reason for the request, a classified reason code, and key personal details of the customer. 
"""
user_prompt =""" 
  Hello, my name is Mateo Gomez. I lost my Credit card on August 17th, and I would like to request its cancellation. The last purchase I made was of a Chicken parmigiana dish at Contoso Restaurant, located near the Hollywood Museum, for $40. Below is my personal information for validation:
  Profession: Accountant
  Social Security Number is 123-45-6789
  Date of birth: 9-9-1989
  Phone number: 949-555-0110
  Personal address: 1234 Hollywood Boulevard Los Angeles CA
  Linked email account: mateo@contosorestaurant.com
  Swift code: CHASUS33XXX
  """
print_response(system_prompt=system_prompt, user_prompt=user_prompt)

# Can we do better?

In [ ]:
system_prompt ="""
Given an input text containing a customer's personal information and request details, extract and structure the relevant information into a JSON object. 
The object should include the reason for the request, a classified reason code, and key personal details of the customer. 

* Expected completion:

  {
      "reason": "Lost card",
      "classified_reason": "lost_card",
      "name": "Mateo Gomez",
      "ssn": "123-45-6789",
      "dob": "09/09/1989"
  }

"""
print_response(system_prompt=system_prompt, user_prompt=user_prompt)

In [ ]:
def print_response(system_prompt, user_prompt):
    response = client.chat.completions.create(
        model = os.getenv("AZURE_OPENAI_COMPLETION_MODEL"),
        messages = [{"role" : "system", "content" : system_prompt}, {"role" : "user", "content" : user_prompt}],
        response_format={"type": "json_object"}

    )
    print(response.choices[0].message.content)

print_response(system_prompt=system_prompt, user_prompt=user_prompt)

## Challenge: Can you write a prompt that generates the expected completion?

In [ ]:
system_prompt ="""Extract data from the given text"""
print_response(system_prompt=system_prompt, user_prompt=user_prompt)


# Can we do even better?

Expected completion:
  ```
  {
      "reason": "Lost card",
      "classified_reason": "lost_card",
      "name": "Mateo Gomez",
      "ssn": "123-45-6789",
      "dob": "09/09/1989"
  }
  ```



  ## Structured Outputs
  
  Structured Outputs is the evolution of JSON mode. While both ensure valid JSON is produced, only Structured Outputs ensure schema adherance. Both Structured Outputs and JSON mode are supported in the Responses API,Chat Completions API, Assistants API, Fine-tuning API and Batch API.

We recommend always using Structured Outputs instead of JSON mode when possible. This will help future-proof your integration and make it easier to work with the API.

| Feature              | Structured Outputs                                      | JSON Mode                                   |
|-----------------------|---------------------------------------------------------|--------------------------------------------|
| Outputs valid JSON    | Yes                                                     | Yes                                        |
| Adheres to schema     | Yes (see supported schemas)                             | No                                         |
| Compatible models     | gpt-4o-mini, gpt-4o-2024-08-06, and later               | gpt-3.5-turbo, gpt-4-*, and gpt-4o-* models |
| Enabling              | `text: { format: { type: "json_schema", "strict": true, "schema": ... } }` | `text: { format: { type: "json_object" } }` |

In [ ]:
schema = {
    "type": "json_schema",
    "json_schema": {
        "name": "my_schema",
        "schema": {
            "type": "object",
            "properties": {
                "reason": {
                    "type": "string"
                },
                "classified_reason": {
                    "type": "string"
                },
                "name": {
                    "type": "string"
                },
                "ssn": {
                    "type": "string"
                },
                "dob": {
                    "type": "string"
                }
            },
            "required": ["reason", "classified_reason", "name", "ssn", "dob"],
        }
    }
}

# Task
Use the schema to validate JSON data using structured output. 